In [ ]:
# File: noodle.ipynb
# Code: Claude Code and Codex
# Review: Ryoichi Ando (ryoichi.ando@zozo.com)
# License: Apache v2.0

In [ ]:
import numpy as np
from frontend import App

# create an app
app = App.create("noodle")

# create a 1D rod strand mesh (long vertical line)
V, E = app.mesh.line([0, 0.01, 0], [0.01, 15, 0], 960)
app.asset.add.rod("strand", V, E)

# create a scene
scene = app.scene.create()

# add invisible hemisphere bowl to collect noodles
scene.add.invisible.sphere([0, 1, 0], 1.1).invert().hemisphere()

# add grid of noodle strands falling into bowl
N, scale = 11, 0.05
for i, j in np.ndindex((N, N)):
    x, y = scale * (i - N / 2), scale * (j - N / 2)
    obj = scene.add("strand").at(x, 0, y).jitter()
    # set material properties for flexible noodles
    (
        obj.param.set("bend", 10)
        .set("contact-gap", 3e-3)
        .set("contact-offset", 3e-3)
        .set("friction", 0.05)
    )

# set preview options for close-up view
opts = {"lookat": [0, 0.25, 0], "eyeup": 1.0, "fov": 4}

# compile the scene and report stats
scene = scene.build().report()

# preview the initial scene
scene.preview(options=opts)

In [ ]:
# create a new session with the compiled scene
session = app.session.create(scene)

# set session parameters with air friction to slow falling
(
    session.param.set("frames", 240)
    .set("fix-xz", 1.0)
    .set("isotropic-air-friction", 1e-5)
)

# build this session
session = session.build()

In [ ]:
# start the simulation and live-preview the results
session.start().preview(options=opts)

# also show simulation logs in realtime
session.stream()

In [ ]:
# create an animation from the simulation results
session.animate(options=opts)

In [ ]:
# export the animation to file
session.export.animation()

In [ ]:
# this is for CI
if app.ci:
    assert session.finished()